In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
splitter = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),)
inputs = th.linspace(1,8,8).unsqueeze(0).unsqueeze(1).unsqueeze(-1)
splitter(inputs).view(1,8)

In [ ]:
splitter = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),)
inputs = th.linspace(1,8,8).unsqueeze(0).unsqueeze(1).unsqueeze(-1)
splitter(inputs).view(1,8)

In [ ]:
splitter = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),)
inputs = th.linspace(1,8,8).unsqueeze(0).unsqueeze(1).unsqueeze(-1)
splitter(inputs).view(1,8)

In [ ]:
splitter = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),)
inputs = th.linspace(1,64,64).unsqueeze(0).unsqueeze(1).unsqueeze(-1)
outs = splitter(inputs)
outs.view(1,64)

In [ ]:
outs.view(1,2,32,1)

In [ ]:
outs.shape

In [ ]:
th.cat(outs[:,])

In [ ]:
dummy_splitter = SubsampleSplitter(stride=[2,1],chunk_chans_first=False)
interleaved = invert(dummy_splitter, outs)
interleaved = interleaved.view(*outs.shape)

In [ ]:
dummy_splitter = SubsampleSplitter(stride=[2,1],chunk_chans_first=False)
inverted = dummy_splitter(interleaved.permute(0,2,1,3))
inverted = inverted.view(interleaved.shape)

In [ ]:
assert np.array_equal(var_to_np(inverted), var_to_np(outs))

In [ ]:
interleaved.shape

In [ ]:
outs.shape

In [ ]:
outs.view(1,64)